In [1]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from db import Arret, Vehicule, Ligne, Trajet, Etape

ArgumentError: Mapper Mapper|Trajet|trajet could not assemble any primary key columns for mapped table 'trajet'

In [2]:
engine = create_engine('sqlite:///database.db')

In [3]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.query(Trajet).all()

In [6]:
with engine.connect() as connection:
    for table in engine.table_names():
        print(table)
        print(pd.read_sql_table(table, connection).head())

arret
   id_arret mne_arret             nom_arret
0    671154  BANCSTMA              BANCHAIS
1    671157  BARONNER     ST AUBIN LA SALLE
2    671375  DEPOT SD  SORTIE DEPOT ST BART
3    671064     2AARD     AVRILLE - ARDENNE
4    671723  MATECHNO  MAISON DE TECHNOPOLE
bus
  etat_vehicule  id_vehicule type_vehicule
0          TDEP    268436139     OmniArtE6
1          TDEP    268435909      OMNICITY
2          HLPS    268436259     OmniSTDE6
3          TDEP    268436472       CITADIS
4          TDEP    268436070      MAN City
etape
   ecart heure_arret_theorique  id_arret    record_timestamp
0      0   2018-06-19 13:49:58    671154 2018-06-19 13:49:58
1      0   2018-06-19 13:51:37    671157 2018-06-19 13:51:37
2      0   2018-06-19 12:55:13    671375 2018-06-19 12:55:13
3      0   2018-06-19 13:52:35    671064 2018-06-19 13:52:35
4      0   2018-06-19 13:53:33    671723 2018-06-19 13:53:33
ligne
   id_ligne                        nom_ligne num_ligne
0    671154    ST SYLVAIN BANCHAIS 

In [17]:
def get_latlon():
    with engine.connect() as connection:
        df = pd.read_sql_table('trajet', connection)
    
    return df[['destination', 'latitude', 'longitude']]

df = get_latlon()
df

,destination,latitude,longitude
0,TRELAZE QUANTINIERE,47.485885,-0.512530
1,VERNUSSON PAR TRELAZE,47.489367,-0.503753
2,PAS EN SERVICE,47.476323,-0.492809
3,ANGERS ROSERAIE,47.511606,-0.596008
4,MONPLAISIR,47.478561,-0.605686
5,ST LEGER LA COUDRE,47.464963,-0.559128
6,BELLE BEILLE,47.491103,-0.528307
7,PAS EN SERVICE,47.400803,-0.542559
8,ESPACE ANJOU,47.496769,-0.514985
9,LAC DE MAINE - L'HOIRIE,47.472852,-0.555017


In [9]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [10]:
def get_mapbox_access_token(folderpath='.', filename="../mapbox.txt"):
    import os
    
    with open(os.path.join(folderpath, filename), 'r') as file:
        s = file.read()
    
    return s

In [11]:
mapbox_access_token = get_mapbox_access_token()

In [26]:
df['latitude'][1]

47.489367

In [27]:
df['longitude'][1]

-0.50375319

In [29]:
data = [
    go.Scattermapbox(
        lat=df['latitude'],
        lon=df['longitude'],
        mode='markers',
        marker=dict(
            size=9
        ),
        text=df['destination'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=df['latitude'][1],
            lon=df['longitude'][1]
        ),
        pitch=0,
        zoom=10
    ),
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='Multiple Mapbox')